In [1]:
import pennylane as qml
from qiskit_quantuminspire.qi_provider import QIProvider
from pennylane_quantuminspire.qi_device import QIDevice
from pennylane.qinfo import fidelity
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [2]:
def datos(n=50):
    np.random.seed(0)
    x_izq = np.random.uniform(-1, -0.1, (n//2, 1))  # clase 0
    x_der = np.random.uniform(0.1, 1, (n//2, 1))    # clase 1
    x = np.vstack([x_izq, x_der])
    y = np.array([0] * (n//2) + [1] * (n//2))
    return x, y


In [3]:
def graficar_bolitas(x, y, titulo=""):
    colores = ['red' if yi == 0 else 'blue' for yi in y]
    plt.scatter(x[:, 0], np.zeros_like(x[:, 0]), c=colores, edgecolor='k', s=100)
    plt.title(titulo)
    plt.yticks([])
    plt.xlabel("x")
    plt.grid(True)
    plt.show()


In [4]:
provider = QIProvider()
backend = provider.get_backend('Tuna-5')
demo_device = QIDevice(backend)


In [ ]:
dev = demo_device

@qml.qnode(dev)
def dru_circuit(params, x):
    for p in params:
        qml.RX(x[0], wires=0)
        qml.RY(p[0] * x[0] + p[1], wires=0)
        qml.RZ(p[2], wires=0)
    return qml.probs(wires=0)


In [6]:
def predecir(x, params):
    probas = dru_circuit(params, x)
    return int(np.argmax(probas))  # clase 0 o 1


In [7]:
def costo(params, x, y):
    loss = 0
    for xi, yi in zip(x, y):
        probas = dru_circuit(params, xi)
        p = probas[yi]
        loss -= np.log(p + 1e-8)  # para evitar log(0)
    return loss / len(x)


In [8]:
def entrenar(params, x_train, y_train, pasos=50, lr=0.2):
    opt = qml.GradientDescentOptimizer(lr)
    historial = []

    for paso in range(pasos):
        params = opt.step(lambda p: costo(p, x_train, y_train), params)
        loss = costo(params, x_train, y_train)
        historial.append(loss)
        print(f"Paso {paso+1}: Costo = {loss:.4f}")
    
    return params, historial


In [ ]:

x_train, y_train = datos(10)


np.random.seed(1)
params = np.random.randn(3)

# Entrenar
params_entrenados, historial = entrenar(params, x_train, y_train, pasos=2, lr=0.2)


DeviceError: 'Timeout while waiting for job .'

In [ ]:
# Predicciones finales
predicciones = np.array([predecir(x, params_entrenados) for x in x_train])
exactitud = np.mean(predicciones == y_train)
print("Exactitud (entrenado):", exactitud)

graficar_bolitas(x_train, predicciones, "Predicción tras entrenamiento")
